In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [4]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [5]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.math.subtract(y_pred, 42)
    b = dist

    loss = tf.math.add(a, b)
    return loss

In [6]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21)(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [7]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mae'])

In [8]:
hist = autoencoder.fit(X, X, epochs=1000, batch_size=32)

Epoch 1/1000
13/13 [==============================] - 1s 2ms/step - loss: 9.8216 - mae: 0.4098
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 3.7039 - mae: 0.3780
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2929 - mae: 0.3703
Epoch 4/1000
13/13 [==============================] - 0s 3ms/step - loss: -0.1733 - mae: 0.3895
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.2608 - mae: 0.3986
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.3012 - mae: 0.3976
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.3219 - mae: 0.3950
Epoch 8/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.3127 - mae: 0.3955
Epoch 9/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.3148 - mae: 0.3945
Epoch 10/1000
13/13 [==============================] - 0s 2ms/step - loss: -0.3184 - mae: 0.3955
Epoch 11/1000
13/13 [=====================

In [20]:
X.shape

(395, 21)

In [19]:
gen = autoencoder.predict(X)

13/13 [==============================] - 0s 1ms/step


In [13]:
dnn_model.predict(gen).min()

13/13 [==============================] - 0s 1ms/step


27.299343

In [23]:
latest = np.array([X[-1]])

In [24]:
latest.shape

(1, 21)

In [25]:
latest_gen = autoencoder.predict(latest)

1/1 [==============================] - 0s 19ms/step


In [28]:
dist = np.sqrt(np.sum(np.square(latest-latest_gen)))
print(dist)

1.8678781254907475


In [11]:
error

NameError: name 'error' is not defined

In [29]:
gen_en = encoder.predict(X)
pd_gen_en = pd.DataFrame(gen_en)
pd_gen_en.to_excel('ae_gen_en.xlsx')

13/13 [==============================] - 0s 1ms/step


In [ ]:
error

NameError: name 'error' is not defined

In [30]:
gen_pop = dnn_model.predict(gen)
pd_gen_pop = pd.DataFrame(gen_pop)
pd_gen_pop.to_excel('ae_gen_pop.xlsx')

13/13 [==============================] - 0s 1ms/step


In [31]:
inverse_gen = scaler.inverse_transform(gen)
pd_gen = pd.DataFrame(inverse_gen)
pd_gen.to_excel('ae_gen.xlsx')